In [24]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [8]:
df= pd.read_csv('le_pen.csv')

In [9]:
df

,name,follower_count,tweet_text,fav_count,rt_count,time
0,Marine Le Pen,2623050,Les annonces du Premier ministre en matière de...,446,142,2021-09-30 19:00:31
1,Marine Le Pen,2623050,🐓 Très heureuse de poursuivre mes rencontres a...,368,142,2021-09-30 12:39:09
2,Marine Le Pen,2623050,Les taxes représentent plus de 60% du prix de ...,565,240,2021-09-30 12:37:54
3,Marine Le Pen,2623050,🤝 À la rencontre des commerçants et des França...,320,126,2021-09-30 12:36:37
4,Marine Le Pen,2623050,"📹 ""On doit engager un bras de fer avec les pay...",537,244,2021-09-30 09:46:55
...,...,...,...,...,...,...
2622,Marine Le Pen,2623052,"""L'immigration de masse touche votre région et...",229,109,2019-04-20 15:26:09
2623,Marine Le Pen,2623052,"""En silence, sans rien demander à personne, sa...",291,190,2019-04-20 15:25:27
2624,Marine Le Pen,2623052,"""L’immigration et ses corollaires le communaut...",140,79,2019-04-20 15:24:42
2625,Marine Le Pen,2623052,"""Je sais qu‘avec sa faconde et son inlassable ...",138,61,2019-04-20 15:24:23


In [138]:
text_sample= df['tweet_text'][0:25]
text_sample[1]

"🐓 Très heureuse de poursuivre mes rencontres avec les Français, aux côtés de femmes et d'hommes de terrain à l'image de @diaz_edwige et @JPGarraud. #Cavignac #MLaFrance\n\n👉🏻 https://t.co/b7AyimkIpR https://t.co/66tgKsZbtU"

In [25]:
stopwords= set(stopwords.words('french'))
punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', text)

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    #tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    #tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet= deEmojify(tweet)
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    #tweet = re.sub('📝 …', '', tweet)
    return tweet


def preprocess_tweet(tweet):
    """Main master function to clean tweets with tokenization and removal of stopwords"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    #tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    #tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    #tweet = re.sub('📝 …', '', tweet)
    tweet=deEmojify(tweet)
    words = nltk.word_tokenize(tweet)
    lemmatized = ' '.join([WordNetLemmatizer().lemmatize(w) for w in words if w not in stopwords])
    return lemmatized



In [26]:
df['preprocessed_tweet']= df['tweet_text'].apply(preprocess_tweet)
df['basic_cleaning']= df['tweet_text'].apply(basic_clean)

In [27]:
df.to_csv('clean_lepen_tweets3.csv',index=False)

In [28]:
df_clean = pd.read_csv('clean_lepen_tweets2.csv' , sep=',', encoding= 'utf-8', engine='python')
df_1= pd.read_csv('clean_lepen_tweets3.csv' , sep=',', encoding= 'utf-8', engine='python')

In [19]:
pd.set_option("display.max_colwidth", -1)
df_clean.head(5)

<ipython-input-19-fe061f0f4977>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


,name,follower_count,tweet_text,fav_count,rt_count,time,preprocessed_tweet,basic_cleaning
0,Marine Le Pen,2623050,"Les annonces du Premier ministre en matière de pouvoir d'achat sont un cautère sur une jambe de bois. Les Français n'ont pas besoin de mesurettes électoralistes, mais d'une baisse réelle et durable du prix des énergies et de l'essence !\nMLP https://t.co/D3jFEKXcqZ",446,142,2021-09-30 19:00:31,annonces premier ministre matière pouvoir achat cautère jambe bois français besoin mesurettes électoralistes baisse réelle durable prix énergies essence mlp,les annonces du premier ministre en matière de pouvoir d achat sont un cautère sur une jambe de bois les français n ont pas besoin de mesurettes électoralistes mais d une baisse réelle et durable du prix des énergies et de l essence mlp
1,Marine Le Pen,2623050,"🐓 Très heureuse de poursuivre mes rencontres avec les Français, aux côtés de femmes et d'hommes de terrain à l'image de @diaz_edwige et @JPGarraud. #Cavignac #MLaFrance\n\n👉🏻 https://t.co/b7AyimkIpR https://t.co/66tgKsZbtU",368,142,2021-09-30 12:39:09,très heureuse poursuivre rencontres français côtés femmes hommes terrain image,très heureuse de poursuivre mes rencontres avec les français aux côtés de femmes et d hommes de terrain à l image de et
2,Marine Le Pen,2623050,Les taxes représentent plus de 60% du prix de l'énergie. Il y a quelque chose d’inique à voir l'État augmenter ses bénéfices quand l'explosion des tarifs pèse lourdement sur le quotidien des Français. #Cavignac #MLaFrance https://t.co/RTusKIIGPZ,565,240,2021-09-30 12:37:54,tax représentent plus prix énergie a quelque chose ’ inique voir état augmenter bénéfices quand explosion tarifs pèse lourdement quotidien français,les taxes représentent plus de du prix de l énergie il y a quelque chose d’inique à voir l état augmenter ses bénéfices quand l explosion des tarifs pèse lourdement sur le quotidien des français
3,Marine Le Pen,2623050,🤝 À la rencontre des commerçants et des Français au marché de #Cavignac. \n\nLes inquiétudes liées au pouvoir d’achat sont nombreuses et légitimes. \n\nLa nouvelle hausse annoncée au 1er octobre des tarifs du gaz doit alerter tous les responsables politiques. #MLaFrance https://t.co/DEBLdaQLeF,320,126,2021-09-30 12:36:37,rencontre commerçants français marché inquiétudes liées pouvoir ’ achat nombreuses légitimes nouvelle hausse annoncée er octobre tarifs gaz doit alerter tous responsables politiques,à la rencontre des commerçants et des français au marché de les inquiétudes liées au pouvoir d’achat sont nombreuses et légitimes la nouvelle hausse annoncée au er octobre des tarifs du gaz doit alerter tous les responsables politiques
4,Marine Le Pen,2623050,"📹 ""On doit engager un bras de fer avec les pays qui ne veulent pas reprendre leurs nationaux qui sont clandestins chez nous. On peut limiter les envois d'argent vers le pays d'origine : ces sommes sont supérieures à l'aide au développement !"" @tv7_sudouest #MLaFrance https://t.co/UR2Gw0Tvz5",537,244,2021-09-30 09:46:55,doit engager bra fer pay veulent reprendre leurs nationaux clandestins chez peut limiter envoi argent vers pay origine supérieures aide développement,on doit engager un bras de fer avec les pays qui ne veulent pas reprendre leurs nationaux qui sont clandestins chez nous on peut limiter les envois d argent vers le pays d origine ces sommes sont supérieures à l aide au développement


In [29]:
df_1

,name,follower_count,tweet_text,fav_count,rt_count,time,preprocessed_tweet,basic_cleaning
0,Marine Le Pen,2623050,"Les annonces du Premier ministre en matière de pouvoir d'achat sont un cautère sur une jambe de bois. Les Français n'ont pas besoin de mesurettes électoralistes, mais d'une baisse réelle et durable du prix des énergies et de l'essence !\nMLP https://t.co/D3jFEKXcqZ",446,142,2021-09-30 19:00:31,annonces premier ministre matière pouvoir achat cautère jambe bois français besoin mesurettes électoralistes baisse réelle durable prix énergies essence mlp,les annonces du premier ministre en matière de pouvoir d achat sont un cautère sur une jambe de bois les français n ont pas besoin de mesurettes électoralistes mais d une baisse réelle et durable du prix des énergies et de l essence mlp
1,Marine Le Pen,2623050,"🐓 Très heureuse de poursuivre mes rencontres avec les Français, aux côtés de femmes et d'hommes de terrain à l'image de @diaz_edwige et @JPGarraud. #Cavignac #MLaFrance\n\n👉🏻 https://t.co/b7AyimkIpR https://t.co/66tgKsZbtU",368,142,2021-09-30 12:39:09,très heureuse poursuivre rencontres français côtés femmes hommes terrain image,très heureuse de poursuivre mes rencontres avec les français aux côtés de femmes et d hommes de terrain à l image de et
2,Marine Le Pen,2623050,Les taxes représentent plus de 60% du prix de l'énergie. Il y a quelque chose d’inique à voir l'État augmenter ses bénéfices quand l'explosion des tarifs pèse lourdement sur le quotidien des Français. #Cavignac #MLaFrance https://t.co/RTusKIIGPZ,565,240,2021-09-30 12:37:54,tax représentent plus prix énergie a quelque chose ’ inique voir état augmenter bénéfices quand explosion tarifs pèse lourdement quotidien français,les taxes représentent plus de du prix de l énergie il y a quelque chose d’inique à voir l état augmenter ses bénéfices quand l explosion des tarifs pèse lourdement sur le quotidien des français
3,Marine Le Pen,2623050,🤝 À la rencontre des commerçants et des Français au marché de #Cavignac. \n\nLes inquiétudes liées au pouvoir d’achat sont nombreuses et légitimes. \n\nLa nouvelle hausse annoncée au 1er octobre des tarifs du gaz doit alerter tous les responsables politiques. #MLaFrance https://t.co/DEBLdaQLeF,320,126,2021-09-30 12:36:37,rencontre commerçants français marché inquiétudes liées pouvoir ’ achat nombreuses légitimes nouvelle hausse annoncée er octobre tarifs gaz doit alerter tous responsables politiques,à la rencontre des commerçants et des français au marché de les inquiétudes liées au pouvoir d’achat sont nombreuses et légitimes la nouvelle hausse annoncée au er octobre des tarifs du gaz doit alerter tous les responsables politiques
4,Marine Le Pen,2623050,"📹 ""On doit engager un bras de fer avec les pays qui ne veulent pas reprendre leurs nationaux qui sont clandestins chez nous. On peut limiter les envois d'argent vers le pays d'origine : ces sommes sont supérieures à l'aide au développement !"" @tv7_sudouest #MLaFrance https://t.co/UR2Gw0Tvz5",537,244,2021-09-30 09:46:55,doit engager bra fer pay veulent reprendre leurs nationaux clandestins chez peut limiter envoi argent vers pay origine supérieures aide développement,on doit engager un bras de fer avec les pays qui ne veulent pas reprendre leurs nationaux qui sont clandestins chez nous on peut limiter les envois d argent vers le pays d origine ces sommes sont supérieures à l aide au développement
...,...,...,...,...,...,...,...,...
2622,Marine Le Pen,2623052,"""L'immigration de masse touche votre région et parfois votre village puisque nos gendarmeries, nos maisons de retraite, et parfois nos châteaux sont transformés pour accueillir dans la clandestinité ces #migrants dont personne ne veut, en réalité."" #OnArrive #Beaucaire https://t.co/60VN4k19Ue",229,109,2019-04-20 15:26:09,immigration masse touche région parfois village puisque gendarmery maisons retraite parfois châteaux transformés accueillir clandestinité dont personne veut réalité,l immigration de masse touch

In [17]:
df= (df_clean[['name', 'tweet_text', 'preprocessed_tweet', 'basic_cleaning']])

In [19]:
df.head(5)

,name,tweet_text,preprocessed_tweet,basic_cleaning
0,Marine Le Pen,"Les annonces du Premier ministre en matière de pouvoir d'achat sont un cautère sur une jambe de bois. Les Français n'ont pas besoin de mesurettes électoralistes, mais d'une baisse réelle et durable du prix des énergies et de l'essence !\nMLP https://t.co/D3jFEKXcqZ",annonces premier ministre matière pouvoir achat cautère jambe bois français besoin mesurettes électoralistes baisse réelle durable prix énergies essence mlp,les annonces du premier ministre en matière de pouvoir d achat sont un cautère sur une jambe de bois les français n ont pas besoin de mesurettes électoralistes mais d une baisse réelle et durable du prix des énergies et de l essence mlp
1,Marine Le Pen,"🐓 Très heureuse de poursuivre mes rencontres avec les Français, aux côtés de femmes et d'hommes de terrain à l'image de @diaz_edwige et @JPGarraud. #Cavignac #MLaFrance\n\n👉🏻 https://t.co/b7AyimkIpR https://t.co/66tgKsZbtU",très heureuse poursuivre rencontres français côtés femmes hommes terrain image,très heureuse de poursuivre mes rencontres avec les français aux côtés de femmes et d hommes de terrain à l image de et
2,Marine Le Pen,Les taxes représentent plus de 60% du prix de l'énergie. Il y a quelque chose d’inique à voir l'État augmenter ses bénéfices quand l'explosion des tarifs pèse lourdement sur le quotidien des Français. #Cavignac #MLaFrance https://t.co/RTusKIIGPZ,tax représentent plus prix énergie a quelque chose ’ inique voir état augmenter bénéfices quand explosion tarifs pèse lourdement quotidien français,les taxes représentent plus de du prix de l énergie il y a quelque chose d’inique à voir l état augmenter ses bénéfices quand l explosion des tarifs pèse lourdement sur le quotidien des français
3,Marine Le Pen,🤝 À la rencontre des commerçants et des Français au marché de #Cavignac. \n\nLes inquiétudes liées au pouvoir d’achat sont nombreuses et légitimes. \n\nLa nouvelle hausse annoncée au 1er octobre des tarifs du gaz doit alerter tous les responsables politiques. #MLaFrance https://t.co/DEBLdaQLeF,rencontre commerçants français marché inquiétudes liées pouvoir ’ achat nombreuses légitimes nouvelle hausse annoncée er octobre tarifs gaz doit alerter tous responsables politiques,à la rencontre des commerçants et des français au marché de les inquiétudes liées au pouvoir d’achat sont nombreuses et légitimes la nouvelle hausse annoncée au er octobre des tarifs du gaz doit alerter tous les responsables politiques
4,Marine Le Pen,"📹 ""On doit engager un bras de fer avec les pays qui ne veulent pas reprendre leurs nationaux qui sont clandestins chez nous. On peut limiter les envois d'argent vers le pays d'origine : ces sommes sont supérieures à l'aide au développement !"" @tv7_sudouest #MLaFrance https://t.co/UR2Gw0Tvz5",doit engager bra fer pay veulent reprendre leurs nationaux clandestins chez peut limiter envoi argent vers pay origine supérieures aide développement,on doit engager un bras de fer avec les pays qui ne veulent pas reprendre leurs nationaux qui sont clandestins chez nous on peut limiter les envois d argent vers le pays d origine ces sommes sont supérieures à l aide au développement
